In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from tqdm import tqdm_notebook as tq


In [2]:
# Import data for probabilities
probability_table = pd.read_csv('driver_attention.csv')
driver_impairment_table = pd.read_csv('driver_impairment.csv')

In [3]:
# View data
display(probability_table.tail())

,Impairments,None,Distracted,"Drowsy, sleepy, asleep, fatigued",Exceeded speed limit,Exceeded safe speed but not speed limit,Driving slowly: below speed limit,Driving slowly in relation to other traffic: not below speed limit,Passing on right,Illegal passing,...,Avoiding other vehicle,Avoiding animal,Apparent unfamiliarity with roadway,Apparent unfamiliarity with vehicle,Apparent general inexperience driving,Use of cruise control contributed to late braking,Other,Unknown,NULL (no value),Total
10,Deaf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,Other,51,26,2,7,2,1,0,0,0,...,0,0,1,0,0,0,0,1,0,107
12,Unknown,4,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
13,NULL (no value),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,Total,32966,1893,594,1135,363,171,155,78,59,...,88,7,76,4,13,0,109,9,0,40859


In [4]:
# Find total Crashes
TOTAL_CRASHES = probability_table[:-1]['Total'].sum()
print(TOTAL_CRASHES)

40859


In [5]:
IMPAIRMENTS = probability_table['Impairments']
IMPAIRMENTS

0                        None apparent
1     Drowsy, sleepy, asleep, fatigued
2                        Ill, blackout
3                                Angry
4                Other emotional state
5                    Drugs, medication
6                       Drugs, alcohol
7                  Other illicit drugs
8             Restricted to wheelchair
9      Impaired due to previous injury
10                                Deaf
11                               Other
12                             Unknown
13                     NULL (no value)
14                               Total
Name: Impairments, dtype: object

In [6]:
BEHAVIOR_PROB_DICT = {}
for col in probability_table:
    if (col == 'Impairments') | (col == 'NULL (no value)') | (col == 'Total'):
        next
    else:
        BEHAVIOR_PROB_DICT[col] = (probability_table[col].iloc[14] / TOTAL_CRASHES)

In [7]:
# BEHAVIOR_PROB_DICT

In [ ]:
# Assign each driver a random behavior
driver_behavior = []
items = list(BEHAVIOR_PROB_DICT.items())  # List of tuples of (key,values)


 

for i in tq(range(1000000)):
    assigned_behavior = 0
    np.random.shuffle(items)
    while assigned_behavior == 0:
        for key, value in items:
        #     print(key, ":", value)
            behavior = np.random.binomial(1, value)
            if behavior == 1:
                driver_behavior.append(key)
                assigned_behavior = 1
                break

In [ ]:
len(driver_behavior)

## Assign Impairment based on Behavior

In [ ]:
display(driver_impairment_table.tail())

In [ ]:
driver_impairment = []

In [ ]:
for driver in tq(driver_behavior):
#     print(driver)
#     print(np.array(driver_impairment_table[driver]))
    probs = np.array(driver_impairment_table[driver])
    impairment_dict = {}
    for c, prob in enumerate(probs):
        impairment_dict[driver_impairment_table['Impairments'].iloc[c]] = prob
#     print(impairment_dict)
    
    
    assigned_impairment = 0
    items = list(impairment_dict.items())
    np.random.shuffle(items)
    while assigned_impairment == 0:
        for key, value in items:
        #     print(key, ":", value)
            impairment = np.random.binomial(1, value)
            if impairment == 1:
                driver_impairment.append(key)
                assigned_impairment = 1
                break


In [ ]:
len(driver_impairment)

In [ ]:
# Based on driver behavior select driver impairment